###  세마포어

In [1]:
import threading
import time

class ThreadPool(object):
    def __init__(self): #생성자에서 락킹 지원
        self.active = []
        self.lock = threading.Lock() 
        
    def acquire(self, name): #락을 획득하는 것은 락을 거는 것.
        with self.lock: # 이것을 오토락이라고 함 맨처음 진입하는 애에 대해 락을 획득
            self.active.append(name)
            print("획득: {0} | 스레드풀: {1}".format(name, self.active))
            
    def release(self, name): #락을 풀때 지원.
        with self.lock: 
            self.active.remove(name)
            print("반환: {0} | 스레드풀: {1}".format(name, self.active))         
            
def worker(semaphore, pool):
    with semaphore: #여기서 세마포어의
        name = threading.currentThread().getName() #임계영역으로 3개만 들어가는 것, 들어온 이름 뽑아내기 위해 겟네임
        pool.acquire(name) # 임계영역에 들어가서
        time.sleep(1)# 1초 있다가 임계영역에서
        pool.release(name) #이러면 빠져나온다
            
if __name__ == "__main__":
    
    threads = []
    pool = ThreadPool()
    semaphore = threading.Semaphore(3) #임계영역에 3개 들어갈 수 있게 설정
    for i in range(10):
        t = threading.Thread( target=worker, name="슬드 "+str(i), args=(semaphore,pool))#세마포어와 스레드 풀을 만들어서 넘겨주는것) # 스레딩 만드는 것
        t.start() # 기동 시키고
        threads.append(t) #
        
    for t in threads:
        t.join() #모든 스레드가 종료할 때 까지 메인스레드는 대기탄다

획득: 슬드 0 | 스레드풀: ['슬드 0']
획득: 슬드 1 | 스레드풀: ['슬드 0', '슬드 1']
획득: 슬드 2 | 스레드풀: ['슬드 0', '슬드 1', '슬드 2']
반환: 슬드 0 | 스레드풀: ['슬드 1', '슬드 2']
반환: 슬드 1 | 스레드풀: ['슬드 2']
획득: 슬드 3 | 스레드풀: ['슬드 2', '슬드 3']
획득: 슬드 4 | 스레드풀: ['슬드 2', '슬드 3', '슬드 4']
반환: 슬드 2 | 스레드풀: ['슬드 3', '슬드 4']
획득: 슬드 5 | 스레드풀: ['슬드 3', '슬드 4', '슬드 5']
반환: 슬드 3 | 스레드풀: ['슬드 4', '슬드 5']
반환: 슬드 4 | 스레드풀: ['슬드 5']
획득: 슬드 6 | 스레드풀: ['슬드 5', '슬드 6']
획득: 슬드 7 | 스레드풀: ['슬드 5', '슬드 6', '슬드 7']
반환: 슬드 5 | 스레드풀: ['슬드 6', '슬드 7']
획득: 슬드 8 | 스레드풀: ['슬드 6', '슬드 7', '슬드 8']
반환: 슬드 6 | 스레드풀: ['슬드 7', '슬드 8']
획득: 슬드 9 | 스레드풀: ['슬드 7', '슬드 8', '슬드 9']
반환: 슬드 7 | 스레드풀: ['슬드 8', '슬드 9']
반환: 슬드 8 | 스레드풀: ['슬드 9']
반환: 슬드 9 | 스레드풀: []
